How to convert a Google sheet to a static marketing website

templates google doc?

templates google sheet?

In [ ]:
var util = require('util');
var importer = require('../Core');
var getInfo = importer.import('get google sheet info');

function getTemplates(doc) {
    return getInfo(doc)
        .then(info => {
            console.log(`Loaded doc: ${info.properties.title} ${info.spreadsheetId}`);
            return info.sheets.reduce((obj, worksheet) => {
                worksheet.properties.spreadsheetId = info.spreadsheetId;
                var layoutTitle = worksheet.properties.title.toLowerCase()
                    .replace(/\s*layout|\s*data|\s*template/ig, '')
                    .split(/\s+/ig).join('-');

                if( typeof obj[layoutTitle] == 'undefined' ) obj[layoutTitle] = {template: null, data: null};
                if( worksheet.properties.title.toLowerCase().includes('data') ) obj[layoutTitle].data = worksheet;
                else obj[layoutTitle].template = worksheet;
                return obj;
            }, {})
        })
}
module.exports = getTemplates;


test templates google sheet?


In [ ]:
var assert = require('assert');
var importer = require('../Core');
var getTemplates = importer.import('templates google sheet');

var docsId = '113mVIumItArQ_oXpfDRXP-2Kw2ms4t48oPJ68_p5P8k';

describe('get templates from sheet', () => {
    
    it('should have at least one page', () => {
        return getTemplates(docsId)
            .then(t => {
                console.log(JSON.stringify(Object.keys(t).reduce((obj, cur) => {
                    obj[cur] = {};
                    obj[cur].template = ((t[cur].template || {}).title || '').substr(0, 1000);
                    obj[cur].data = (t[cur].data ? ('yes: ' + t[cur].data.rowCount) : 'no');
                    return obj;
                }, {}), null, 4))
                assert(Object.keys(t).length > 0, 'should have a page called template');
            })
    })
})


get google sheet info?


In [ ]:
var importer = require('../Core');
var authorizeSheets = importer.import('authorize sheets api');

function getInfo(link) {
    var docId = link.replace(/https:\/\/docs.google.com\/spreadsheets\/d\/|\/edit.*$|\/copy.*$/ig, '');
    var sheets;
    
    return authorizeSheets()
        .then(s => sheets = s)
        .then(() => util.promisify(sheets.spreadsheets.get.bind(sheets))({
            spreadsheetId: docId
        }))
        .then(res => res.data)
}

module.exports = getInfo;


get google data sheet?

google sheet array objects?


In [ ]:
var Remarkable = require('remarkable');
var md = new Remarkable({html: true, xhtmlOut: true, breaks: true});
var importer = require('../Core');
var getRows = importer.import('get worksheet rows');

function getDataSheet(link, worksheet) {
    if(typeof link === 'object') {
        worksheet = link;
        link = worksheet.properties.spreadsheetId;
    }
    
    return getRows(link, worksheet)
        .then(rows => {
            var columns = [];
            return rows.reduce((acc, o, i) => {
                if (i == 0) {
                    columns = o;
                    return acc;
                }
                if (!acc[i - 1]) {
                    acc[i - 1] = {};
                }
                o.forEach((cell, j) => {
                    var html = md.render(cell);
                    // if all markdown did was insert a paragraph and line break, use value instead
                    if(html.replace(/<\/?p>|\s+/ig, '') == cell.replace(/<\/?p>|\s+/ig, '')) {
                        html = cell;
                    }
                    acc[i - 1][columns[j]] = html;
                })
                return acc;
            }, []);
        })
}

module.exports = getDataSheet;


get worksheet rows?


In [ ]:
var util = require('util');
var importer = require('../Core');
var authorizeSheets = importer.import('authorize sheets api');
var getInfo = importer.import('get google sheet info');

function getRows(link, worksheet) {
    if(typeof link === 'object') {
        worksheet = link;
        link = worksheet.properties.spreadsheetId;
    }
    
    var docId = link.replace(/https:\/\/docs.google.com\/spreadsheets\/d\/|\/edit.*$|\/copy.*$/ig, '');
    var sheets, wsPromise;
    
    if(typeof worksheet === 'string') {
        wsPromise = getInfo(link).then(info => worksheet = info.sheets.filter(s => s.properties.title === worksheet)[0]);
    } else {
        wsPromise = Promise.resolve(worksheet);
    }

    return authorizeSheets()
        .then(s => sheets = s)
        .then(() => wsPromise)
        .then(() => util.promisify(sheets.spreadsheets.values.get.bind(sheets))({
            spreadsheetId: docId,
            range: `'${worksheet.properties.title}'!A1:Z${worksheet.properties.gridProperties.rowCount}`
        }))
        .then(res => res.data.values)
}

module.exports = getRows;


test google data sheet?


In [ ]:
var assert = require('assert');
var importer = require('../Core');
var getTemplates = importer.import('templates google sheet');
var getDataSheet = importer.import('google sheet array objects');

var docsId = '113mVIumItArQ_oXpfDRXP-2Kw2ms4t48oPJ68_p5P8k';

describe('get data from data sheet', () => {
    
    it('should process at least one data sheet', () => {
        return getTemplates(docsId)
            .then(t => {
                var key = Object.keys(t).filter(k => t[k].data)[0];
                assert(typeof key != 'undefined', 'should have at least one data sheet');
                return getDataSheet(t[key].data);
            })
            .then(data => {
                console.log(data.length);
                console.log(data);
                assert(data.length > 0, 'should have data');
            })
    })
    
    
})


add a row of data to a google sheet?

add row data google sheet?


In [3]:
var importer = require('../Core');
var getInfo = importer.import('get google sheet info');

function addRow(link, newRow) {
    var docId = link.replace(/https:\/\/docs.google.com\/spreadsheets\/d\/|\/edit.*$|\/copy.*$/ig, '');
    
    return getInfo(docId)
        .then(info => {
            console.log('Loaded doc: ' + info.properties.title);
            sheet = info.sheets[0];
            console.log('sheet #1: ' + sheet.title + ' ' + sheet.gridProperties.rowCount + 'x' + sheet.gridProperties.columnCount);

            return util.promisify(doc.addRow)(1, newRow);
        });
}

module.exports = addRow;

[Function: addRow]

test google sheet add row?


In [ ]:
var assert = require('assert');
var importer = require('../Core');
var addRow = importer.import('add row data google sheet');

var docsId = '16b1Z0sQkYhtMUfP7qMRL3vRBPWQsRbSlntURkMqWGX0';

describe('add row data to google sheet', () => {
    
    it('should add a row', () => {
        return addRow(docsId, {
                timestamp: Date.now(),
                name: 'Test Name',
                email: 'test@test.com',
                subject: 'This is a test subject',
                message: 'This is a new message',
                responded: 0
            })
            .then(rows => {
                assert(typeof rows != 'undefined', 'should have added a row to the sheet');
            })
    })
    
    
})


authorize sheets api?


In [ ]:
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var credentials;
if(fs.existsSync('./sheet to web-8ca5784e0b05.json')) {
    credentials = path.resolve('./sheet to web-8ca5784e0b05.json');
} else {
    credentials = path.join(PROFILE_PATH, '.credentials/sheet to web-8ca5784e0b05.json');
}

var {google} = require('googleapis');
var {auth} = require('google-auth-library');
var GOOGLE_AUTH_SCOPE = [
    'https://www.googleapis.com/auth/spreadsheets'
];

function authorizeSheets() {
    var sheets, client;

    return auth.getClient({
        keyFile: credentials,
        scopes: GOOGLE_AUTH_SCOPE
    })
        .then(c => {
            client = c
            sheets = google.sheets({version: 'v4', auth: client});
        })
        .then(() => sheets)
}

module.exports = authorizeSheets;
